In [1]:
!pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.7 MB/s eta 0:00:00


In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from transformers import TextIteratorStreamer
from threading import Thread
import time

In [3]:
tokenizer = AutoTokenizer.from_pretrained(
    "elyza/ELYZA-japanese-Llama-2-7b-instruct"
)
model = AutoModelForCausalLM.from_pretrained(
    "elyza/ELYZA-japanese-Llama-2-7b-instruct",
    torch_dtype=torch.float16,
    device_map="auto"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

...

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

In [12]:
prompt = """「Buildxを使うことで、異なるアーキテクチャ（例：linux/arm64 や linux/ppc64le など）に向けたイメージもビルド可能で、クロスプラットフォームのビルドに特化しています。」という文章を、中学生にも分かりやすいよう言い換えてください。"""


In [14]:
st = time.time()
with torch.no_grad():
    streamer = TextIteratorStreamer(tokenizer)
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    generation_kwargs = dict(
        input_ids=token_ids.to(model.device),
        max_new_tokens=256,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()
fin = time.time()
print(f"time is :{fin - st:.3f}", end = "")
# 出力
for new_text in streamer:
    if "「" in new_text :
      f = time.time()
      print(f"<<time is :{f - st:.3f}s>>",end = "")
    print(new_text.replace(" ", ""), end="")

time is :0.012<<time is :0.015s>>「Buildxを使うことで、異なるアーキテクチャ（例：linux/arm64やlinux/ppc64leなど）に向けたイメージもビルド可能で、クロスプラットフォームのビルドに特化しています。」という文章を、中学生にも分かりやすいよう言い換えてください。
<<time is :0.751s>>「Buildxを使うことで、違う種類のコンピュータのためのプログラムも作ることができて、異なるアーキテクチャに向けたイメージもビルド可能で、クロスプラットフォームのビルドに特化しています。」</s>